In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
from PIL import Image
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
url = 'https://www.starbucks.co.kr/index.do'
driver.get(url)
time.sleep(3)     # 스타벅스는 바로 매장찾기 부분의 url을 넣어도 튕겨나오지 않음. 이건 복습 겸...

In [3]:
action = ActionChains(driver)

In [4]:
parent_level_menu = driver.find_element_by_class_name("gnb_nav03")
action.move_to_element(parent_level_menu).perform()
time.sleep(0.5)

xpath = '//*[@id="gnb"]/div/nav/div/ul/li[3]/div/div[1]/div/ul[1]/li[3]/a'

child_level_menu = driver.find_element_by_xpath(xpath)
child_level_menu.click()

time.sleep(1)

In [5]:
xpath = '''//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a'''

seoul = driver.find_element_by_xpath(xpath)
seoul.click()
time.sleep(1)

In [6]:
xpath = '''//*[@id="mCSB_2_container"]/ul/li[1]/a'''

whole = driver.find_element_by_xpath(xpath)
whole.click()
time.sleep(1)   # 서울 전체 매장 검색. xpath 말고 css selector로 해도 됨.

* 스크린샷을 찍고 싶다면

driver.save_screenshot('opinet1.png')

Image.open('opinet1.png')

#### HTML 분석하기 (자동으로 시작해 수동으로 넘어감)

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#### 서울 매장 리스트 추출

li.quickResultLstCon

In [8]:
starbucks_soup_list = soup.select('li.quickResultLstCon')
len(starbucks_soup_list)

585

#### 첫번째 매장 정보 출력

In [9]:
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

#### 매장 이름 / 주소 / 위도 / 경도 / 전화번호 / 타입 추출하기

* 매장 이름

In [10]:
starbucks_store = starbucks_soup_list[0]

In [11]:
name = starbucks_store.select('strong')[0].text     # 첫 번째 '역삼아레나빌딩'은 <strong 부분에 있음. strong 태그.
print(name)

역삼아레나빌딩  


* 위도/경도

In [12]:
lat = starbucks_store['data-lat']
print(lat)

37.501087


In [13]:
long = starbucks_store['data-long']
print(long)                   # A = "B" 형식. dict 형식이라 쉽게 찾을 수 있음.

127.043069


* 매장 주소 & 전화번호

In [14]:
starbucks_store.select('p')[0].text

'서울특별시 강남구 언주로 425 (역삼동)1522-3232'

In [15]:
address = str(starbucks_store.select('p')[0]).split('<br/>')[0].split('>')[1] 
print(address)                              # 계속 문자열을 보고, <> 기준으로 쪼개줘야 함. 가장 어려운 파트.

서울특별시 강남구 언주로 425 (역삼동)


In [16]:
number = str(starbucks_store.select('p')[0]).split('<br/>')[1].split('<')[0] 
print(number)   

1522-3232


* 매장 타입 찾기

** 스타벅스의 특성 상, 보이는 '리저브 2호'가 진짜 타입이 아닌 pin_ 뒤에 붙는 것이 진짜 타입임. pin 부분을 따로 추출해야 함.

In [17]:
storetype = starbucks_store.select('i')[0]['class'][0][4:]
print(storetype)

general


* 반복문

storename = i.select('strong')[0].text
storelat = i['data-lat']
storelong = i['data-long']
storeaddress = str(i.select('p')[0]).split('<br/>')[0].split('>')[1] 
storenumber = str(i.select('p')[0]).split('<br/>')[1].split('<')[0] 
storetype = i.select('i')[0]['class'][0][4:]

* 기존의 starbucks_store는 샘플용 파일이라...

In [18]:
# 초기변수 선언
starbucks_list = []

for i in starbucks_soup_list:
    storename = i.select('strong')[0].text
    storelat = i['data-lat']
    storelong = i['data-long']
    storeaddress = str(i.select('p')[0]).split('<br/>')[0].split('>')[1] 
    storenumber = str(i.select('p')[0]).split('<br/>')[1].split('<')[0] 
    storetype = i.select('i')[0]['class'][0][4:]
    starbucks_list.append([storename,storelat,storelong,storeaddress,storenumber,storetype])
    

In [19]:
columns = ['매장이름', '매장위도', '매장경도', '매장주소', '매장번호', '매장타입']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head(3)

,매장이름,매장위도,매장경도,매장주소,매장번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general


In [20]:
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585 entries, 0 to 584
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장이름    585 non-null    object
 1   매장위도    585 non-null    object
 2   매장경도    585 non-null    object
 3   매장주소    585 non-null    object
 4   매장번호    585 non-null    object
 5   매장타입    585 non-null    object
dtypes: object(6)
memory usage: 27.5+ KB


In [21]:
seoul_starbucks_df.to_excel('seoul_starbucks_list.xlsx', index = False)

In [22]:
driver.close()

## 다운받은 엑셀 파일 이용한 작업

#### 서울시의 인구 데이터 & 기업 데이터

In [23]:
!dir report.txt

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 6E22-98FE

 c:\py_source\데이터분석_5_스타벅스입지조건 디렉터리

2022-07-23  오후 10:36             3,175 report.txt
               1개 파일               3,175 바이트
               0개 디렉터리  37,316,063,232 바이트 남음


In [24]:
!dir report2.txt

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 6E22-98FE

 c:\py_source\데이터분석_5_스타벅스입지조건 디렉터리

2022-07-23  오후 10:37            85,865 report2.txt
               1개 파일              85,865 바이트
               0개 디렉터리  37,316,063,232 바이트 남음


In [25]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
from PIL import Image
import warnings
warnings.filterwarnings(action='ignore')

In [26]:
import pandas as pd

In [27]:
sgg_pop_df = pd.read_csv('report.txt', sep = '\t', header=2) 
columns = {
    '기간' : 'GIGAN',
'자치구' : 'JACHIGU',
'세대' : 'SEDAE',
'계' : 'GYE_1',
'남자' : 'NAMJA_1',
'여자' : 'YEOJA_1',
'계.1' : 'GYE_2',
'남자.1' : 'NAMJA_2',
'여자.1' : 'YEOJA_2',
'계.2' : 'GYE_3',
'남자.2' : 'NAMJA_3',
'여자.2' : 'YEOJA_3',
'세대당인구' : 'SEDEDANGINGU',
'65세이상고령자' : 'N_65SEISANGGORYEONGJA'
}

sgg_pop_df.rename(columns=columns, inplace = True)


# CSV : Commar Separate Value. ,로 구분되어 있음.
# TSV : Tab 키로 구분되어 있음. SSV : Space Bar 키로 구분되어 있음. 이 문서는 TSV.
# 이스케이프 문자 : \n(줄바꿈) / \t(탭 키 = 스페이스 바 4칸) / \r(커서를 줄 맨 앞으로 이동)
# 3번째 행이 제목. 즉 그 아래부터가 내용.

#### 정보 요약 출력 info()

In [28]:
sgg_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  26 non-null     object 
 1   JACHIGU                26 non-null     object 
 2   SEDAE                  26 non-null     object 
 3   GYE_1                  26 non-null     object 
 4   NAMJA_1                26 non-null     object 
 5   YEOJA_1                26 non-null     object 
 6   GYE_2                  26 non-null     object 
 7   NAMJA_2                26 non-null     object 
 8   YEOJA_2                26 non-null     object 
 9   GYE_3                  26 non-null     object 
 10  NAMJA_3                26 non-null     object 
 11  YEOJA_3                26 non-null     object 
 12  SEDEDANGINGU           26 non-null     float64
 13  N_65SEISANGGORYEONGJA  26 non-null     object 
dtypes: float64(1), object(13)
memory usage: 3.0+ KB


In [29]:
sgg_pop_df.head()    # 합계는 사실 필요없음. 분석에 필요한 데이터만 추출하기.

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDEDANGINGU,N_65SEISANGGORYEONGJA
0,2022.1/4,합계,"4,451,444","9,732,617","4,716,704","5,015,913","9,506,778","4,613,910","4,892,868","225,839","102,794","123,045",2.14,"1,628,980"
1,2022.1/4,종로구,"73,766","153,684","74,018","79,666","144,275","69,911","74,364","9,409","4,107","5,302",1.96,"28,064"
2,2022.1/4,중구,"63,767","131,450","63,869","67,581","122,173","59,448","62,725","9,277","4,421","4,856",1.92,"24,742"
3,2022.1/4,용산구,"111,207","236,518","114,571","121,947","222,169","106,735","115,434","14,349","7,836","6,513",2.00,"39,461"
4,2022.1/4,성동구,"134,533","291,609","141,662","149,947","285,073","138,847","146,226","6,536","2,815","3,721",2.12,"46,896"


In [30]:
condition = sgg_pop_df['JACHIGU'] != '합계'
sgg_pop_df_selected = sgg_pop_df[condition]
sgg_pop_df_selected.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDEDANGINGU,N_65SEISANGGORYEONGJA
1,2022.1/4,종로구,"73,766","153,684","74,018","79,666","144,275","69,911","74,364","9,409","4,107","5,302",1.96,"28,064"
2,2022.1/4,중구,"63,767","131,450","63,869","67,581","122,173","59,448","62,725","9,277","4,421","4,856",1.92,"24,742"
3,2022.1/4,용산구,"111,207","236,518","114,571","121,947","222,169","106,735","115,434","14,349","7,836","6,513",2.00,"39,461"
4,2022.1/4,성동구,"134,533","291,609","141,662","149,947","285,073","138,847","146,226","6,536","2,815","3,721",2.12,"46,896"
5,2022.1/4,광진구,"169,483","353,110","169,525","183,585","340,707","164,346","176,361","12,403","5,179","7,224",2.01,"52,604"


In [31]:
sgg_pop_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  25 non-null     object 
 1   JACHIGU                25 non-null     object 
 2   SEDAE                  25 non-null     object 
 3   GYE_1                  25 non-null     object 
 4   NAMJA_1                25 non-null     object 
 5   YEOJA_1                25 non-null     object 
 6   GYE_2                  25 non-null     object 
 7   NAMJA_2                25 non-null     object 
 8   YEOJA_2                25 non-null     object 
 9   GYE_3                  25 non-null     object 
 10  NAMJA_3                25 non-null     object 
 11  YEOJA_3                25 non-null     object 
 12  SEDEDANGINGU           25 non-null     float64
 13  N_65SEISANGGORYEONGJA  25 non-null     object 
dtypes: float64(1), object(13)
memory usage: 2.9+ KB


In [32]:
columns = ['JACHIGU', 'GYE_1']   # 분석에 필요한 건 저 두 개 뿐.
sgg_pop_df_final = sgg_pop_df_selected[columns]
sgg_pop_df_final.head()

,JACHIGU,GYE_1
1,종로구,"153,684"
2,중구,"131,450"
3,용산구,"236,518"
4,성동구,"291,609"
5,광진구,"353,110"


In [33]:
sgg_pop_df_final.columns = ['시군구명','주민등록인구']

In [34]:
sgg_pop_df_final.head()

,시군구명,주민등록인구
1,종로구,"153,684"
2,중구,"131,450"
3,용산구,"236,518"
4,성동구,"291,609"
5,광진구,"353,110"


In [35]:
sgg_pop_df_final.to_excel('sgg_pop.xlsx', index = False)

In [36]:
!dir sgg_pop.xlsx

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 6E22-98FE

 c:\py_source\데이터분석_5_스타벅스입지조건 디렉터리

2022-07-26  오후 01:51             6,086 sgg_pop.xlsx
               1개 파일               6,086 바이트
               0개 디렉터리  37,315,854,336 바이트 남음


#### 사업체 데이터

In [37]:
sgg_biz_df = pd.read_csv('report2.txt', sep='\t',header = 2)
columns = {
    '기간' : 'GIGAN',
    '자치구' : 'JACHIGU',
    '동' : 'DONG',
    '사업체수' : 'SAUPCHESU',
    '여성대표자' : 'YS',
    '계' : 'GYE',
    '남' : 'NAM',
    '여' : 'YEO'
}

sgg_biz_df.rename(columns = columns, inplace = True)
sgg_biz_df.head()

,GIGAN,JACHIGU,DONG,SAUPCHESU,YS,GYE,NAM,YEO,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
0,2019,합계,합계,"823,624","281,318","5,226,997","2,885,614","2,341,383",24,426,...,"1,296","141,582","35,761","351,320","29,168","400,143","23,527","92,752","69,345","189,557"
1,2019,종로구,소계,"39,679","13,356","260,446","146,392","114,054",3,18,...,94,"15,109",829,"13,474",750,"17,366",899,"6,034","2,439","7,649"
2,2019,종로구,사직동,"3,574","1,203","47,522","25,154","22,368",1,12,...,23,"6,088",83,786,106,"1,096",91,"1,653",253,906
3,2019,종로구,삼청동,732,327,"4,601","2,271","2,330",-,-,...,4,"1,291",17,297,7,30,19,185,25,194
4,2019,종로구,부암동,599,255,"3,454","1,764","1,690",-,-,...,3,60,38,"1,190",19,183,18,49,82,325


In [38]:
condition = sgg_biz_df['DONG'] == '소계'   # '소계'를 사용할 예정이고, 소계는 DONG에 있음.
sgg_biz_df_selected = sgg_biz_df[condition]
sgg_biz_df_selected.head()

,GIGAN,JACHIGU,DONG,SAUPCHESU,YS,GYE,NAM,YEO,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
1,2019,종로구,소계,"39,679","13,356","260,446","146,392","114,054",3,18,...,94,"15,109",829,"13,474",750,"17,366",899,"6,034","2,439","7,649"
19,2019,중구,소계,"60,127","22,451","392,568","213,484","179,084",-,-,...,65,"20,287",502,"8,598",760,"10,342",558,"3,637","2,112","8,090"
35,2019,용산구,소계,"20,254","7,074","135,240","74,570","60,670",-,-,...,47,"3,101",537,"6,973",562,"7,147",401,"2,623","1,591","5,494"
52,2019,성동구,소계,"28,343","8,563","178,243","102,769","75,474",2,5,...,45,"4,127",995,"11,678",754,"9,255",592,"2,113","2,079","7,787"
70,2019,광진구,소계,"24,445","8,921","126,305","67,899","58,406",-,-,...,36,"2,678","1,209","14,184",916,"11,551","1,052","4,209","2,552","5,828"


In [39]:
columns = ['JACHIGU', 'GYE', 'SAUPCHESU']
sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
1,종로구,"260,446","39,679"
19,중구,"392,568","60,127"
35,용산구,"135,240","20,254"
52,성동구,"178,243","28,343"
70,광진구,"126,305","24,445"


* 인덱스 재정렬

In [40]:
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop=True)

In [41]:
sgg_biz_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구명    25 non-null     object
 1   종사자수    25 non-null     object
 2   사업체수    25 non-null     object
dtypes: object(3)
memory usage: 728.0+ bytes


In [42]:
sgg_biz_df_final.to_excel('sgg_biz.xlsx', index = False)

In [43]:
!dir sgg_biz.xlsx

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 6E22-98FE

 c:\py_source\데이터분석_5_스타벅스입지조건 디렉터리

2022-07-26  오후 01:52             6,330 sgg_biz.xlsx
               1개 파일               6,330 바이트
               0개 디렉터리  37,315,784,704 바이트 남음


### 스타벅스 매장 경로 파일 불러오기

In [44]:
import pandas as pd

In [45]:
seoul_starbucks = pd.read_excel('seoul_starbucks_list.xlsx', header = 0)
seoul_starbucks.head()

,매장이름,매장위도,매장경도,매장주소,매장번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve


* 스타벅스 문서와 인구/사업체 수를 join하기 위해 공통 부분인 '시군구명' 만들기.

In [46]:
sgg_names = []

for i in seoul_starbucks['매장주소']:
    sgg = i.split()[1]
    sgg_names.append(sgg)
    
seoul_starbucks['시군구명'] = sgg_names

seoul_starbucks.head()

,매장이름,매장위도,매장경도,매장주소,매장번호,매장타입,시군구명
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general,강남구
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general,강남구
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general,강남구
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general,강남구
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve,강남구


* 시군구 목록 데이터 불러오기 (미리 준비됨)

In [48]:
seoul_sgg = pd.read_excel('seoul_sgg_list.xlsx')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155


#### 시군구별 스타벅스 매장 수 카운트

In [49]:
starbucks_sgg_count = seoul_starbucks.pivot_table(index = '시군구명',
                                                  values = '매장이름',
                                                  aggfunc = 'count').rename(columns={'매장이름' : '스타벅스_매장 수'})

starbucks_sgg_count.head()

,스타벅스_매장 수
시군구명,
강남구,89
강동구,16
강북구,6
강서구,25
관악구,12


In [50]:
seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how = 'left', on='시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장 수
0,11320,도봉구,37.665861,127.031767,4
1,11380,은평구,37.617612,126.922700,10
2,11230,동대문구,37.583801,127.050700,9
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,13


In [51]:
seoul_sgg_pop = pd.read_excel('sgg_pop.xlsx')
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_pop, how = 'left', on='시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장 수,주민등록인구
0,11320,도봉구,37.665861,127.031767,4,"318,100"
1,11380,은평구,37.617612,126.922700,10,"476,013"
2,11230,동대문구,37.583801,127.050700,9,"352,592"
3,11590,동작구,37.496504,126.944307,11,"394,649"
4,11545,금천구,37.460097,126.900155,13,"244,901"


In [52]:
seoul_sgg_biz = pd.read_excel('sgg_biz.xlsx')
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_biz, how = 'left', on='시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장 수,주민등록인구,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,4,"318,100","72,939","18,628"
1,11380,은평구,37.617612,126.922700,10,"476,013","93,631","24,681"
2,11230,동대문구,37.583801,127.050700,9,"352,592","138,400","31,324"
3,11590,동작구,37.496504,126.944307,11,"394,649","106,159","19,793"
4,11545,금천구,37.460097,126.900155,13,"244,901","242,686","33,814"


### 시각화 (예시)

In [53]:
import pandas as pd
import folium
import json
import warnings
warnings.filterwarnings(action='ignore')

In [54]:
hongik = folium.Map(location = [37.5515762, 126.9265196], zoom_start = 19,
                   titles = 'OpenStreetMap')
folium.Marker([37.5515762, 126.9265196], popup = '홍익대학교',
              icon = folium.Icon(color='black', icon='info-sign')).add_to(hongik)

hongik

# icon은 info-sign, home, star, flag, bookmark 등등 여러 종류가 있음.

In [55]:
hongik = folium.Map(location = [37.5515762, 126.9265196], zoom_start = 19,
                   titles = 'OpenStreetMap')

folium.CircleMarker([37.5515762, 126.9265196], popup = '홍익대학교', 
                    radius = 100, color = 'green', fill_color = 1).add_to(hongik)

hongik

## 스타벅스 매장 지도에 출력하기

In [56]:
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장 수,주민등록인구,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,4,"318,100","72,939","18,628"
1,11380,은평구,37.617612,126.922700,10,"476,013","93,631","24,681"
2,11230,동대문구,37.583801,127.050700,9,"352,592","138,400","31,324"
3,11590,동작구,37.496504,126.944307,11,"394,649","106,159","19,793"
4,11545,금천구,37.460097,126.900155,13,"244,901","242,686","33,814"


In [57]:
seoul_starbucks = pd.read_excel('seoul_starbucks_list.xlsx')
seoul_starbucks.head()

,매장이름,매장위도,매장경도,매장주소,매장번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve


In [59]:
starbucks_map = folium.Map(location = [37.5502, 126.984], zoom_start=11,
                          tiles = 'Stamen Terrain')

for i in seoul_starbucks.index:
    lat = seoul_starbucks.loc[i, '매장위도']
    lng = seoul_starbucks.loc[i, '매장경도']
    stype = seoul_starbucks.loc[i,'매장타입']
    fillcolor = ''
    if stype == 'general':
        fillcolor = 'green'
        size = 3
    elif stype == 'reserve':
        fillcolor = 'blue'
        size = 5
    else :
        fillcolor = 'black'
        size = 5
    folium.CircleMarker(location = [lat,lng], fill = True, fill_color = fillcolor, fill_opacity=1,
                        color = 'yellow', weight = 1, radius = size).add_to(starbucks_map)
          
starbucks_map
    
    
# fill = True : 내부/외부 경계선 구분. fill_opacity : 투명도

# 일반 매장은 녹색, 리저브 매장은 푸른색, DT 매장은 검은색으로 표시하고 크기에 차이를 뒀음.

### 아까 만들어 둔 구 별 내용 엑셀 파일 저장 후 가공하기

In [60]:
import pandas as pd
import folium
import json

In [61]:
seoul_sgg.to_excel('seoul_sgg_stat.xlsx', index = False)